In [1]:
import requests
import bs4
import nltk
# Load, explore, process and plot data
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pk

#Text Preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('omw-1.4')

In [ ]:
def get_blog_url(soup):
    div_ = soup.find_all('div', attrs = {'class':'FL PR20'})
    url_list = []
    for title in div_:
        href = title.find('a')['href']
        url_list.append("https://www.moneycontrol.com/"+href)
    return url_list

In [ ]:

def get_blog_content(url):
    request = requests.get(url)
    print('Received URL',url)
    soup = bs4.BeautifulSoup(request.text,'html.parser')
    all_script = soup.find_all('script',attrs = {'type':'application/ld+json'})
    raw_article_str = all_script[2].get_text().replace('\r\n',' ')
    parts = re.split(r"""("[^"]*"|'[^']*')""", raw_article_str)
    parts[::2] = map(lambda s: "".join(s.split()), parts[::2])
    article_str = "".join(parts)
    article_str = article_str[1:]
    article_str = article_str[:-1]
    article_dict = json.loads(article_str, strict=False)
    all_tags = soup.find_all('div',attrs={'class':'tags_first_line'})
    list_all_tags=[]
    for i in all_tags:
        list_all_tags.append(i.get_text())
    tags = list_all_tags[0].replace('Tags','')
    tags = tags.replace('\n','')
    tags = tags.split('#')
    tags = tags[1:]
    tags = ', '.join([str(elem).strip() for elem in tags])
    article_dict['tags'] = tags
    return article_dict

In [ ]:
def get_page_no(url, sc_id, page_no, next, year):
    request = requests.get(url)
    soup = BeautifulSoup(request.text,'html.parser')

    
    all_page_no = soup.find_all('div', attrs = {'class': 'pages MR10 MT15'})
    page_list  = [i.text for i in all_page_no[0].find_all('a')]


    if any(map(str.isdigit,page_list[-1])):
        return int(page_list[-1]), next
    else:
        next = next + 1
        page_no = int(page_list[-2])
        url = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id="+sc_id+"&scat=&pageno="+str(page_no)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
    return get_page_no(url, sc_id, page_no, next, year)
    



In [ ]:
def save_company_data(url_ = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?", sc_id=[], pageno=1, next=0, years=[]):
    for company in sc_id:
        df = pd.DataFrame(columns = ['company', 'datePublished', 'author', 'headline',
                                     'description', 'articleBosy','tags','url'])
        for year in years:
            print('Year: ', year)
            print('Page_No:', pageno)
            print('Next: ', next)

            url = url_ + "sc_id="+company+"&scat=&pageno="+str(pageno)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
            print('url: ', url)

            max_page_no, max_next = get_page_no(url, company, pageno, next, year)
            max_next = max_next + 1
            print('Total Page:',max_page_no)
            for i in range(max_next):
                for j in range((i*10)+1, (i*10)+11):
                    if j <= max_page_no:
                        url_list = []
                        url = url_ + "sc_id="+company+"&scat=&pageno="+str(j)+"&next="+str(i)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
                        print(url)
                        request = requests.get(url)
                        soup = BeautifulSoup(request.text, 'html.parser')
                        
                        url_list = get_blog_url(soup)
                        
                        frame1 = []

                        
                        for url in url_list:
                            try:
                                #print('Blog URL:',url)
                                article_dict = get_blog_content(url)

                                
                                print(company)
                                print(article_dict['datePublished'])
                                print(article_dict['author'])
                                print(article_dict['headline'])
                                print(article_dict['description'])
                                print(article_dict['articleBody'])
                                print(article_dict['tags'])
                                print(article_dict['url'])
                                print('--------------------------------------------------------')

                                article_lst = [[company,
                                                 article_dict['datePublished'],
                                                 article_dict['author'],
                                                 article_dict['headline'],
                                                 article_dict['description'],
                                                 article_dict['articleBody'],
                                                 article_dict['tags'],
                                                 url]]
                               
       
                                df = pd.concat([df,pd.DataFrame(article_lst, columns=['company','datePublished','author','headline',
                                                                                   'description','articleBody','tags','url'])],axis=0, ignore_index = True)
                               

                            except:
                                
                                article_lst = [[company, 'error','error','error','error','error','error',url]]
                                
     
                                df = pd.concat([df,pd.DataFrame(article_lst, columns=['company','datePublished','author','headline',
                                                                                   'description','articleBody','tags','url'])],axis=0, ignore_index = True)
                               
                                continue
                           
        df.to_csv(company+'.csv')

In [ ]:
save_company_data(sc_id=['RI'], years = [2008,2007])

In [2]:
df = pd.read_csv('News_Data/RI(2011_2023).csv')

In [3]:
df.head()
df.columns

Index(['S. No.', 'company', 'datePublished', 'author', 'headline',
       'description', 'tags', 'url', 'articleBody'],
      dtype='object')

In [4]:
df.columns
df = df.drop(['S. No.'], axis=1)

In [5]:
article_sentiments = pd.DataFrame({
                        'neg':[],
                        'neu':[],
                        'pos':[],
                        'compound':[]
})

In [6]:
def preprocessing(sentence):
  sentence = re.sub(r"n\'t", " not", sentence)
  sentence = re.sub(r"\'re", " are", sentence)
  sentence = re.sub(r"\'s", " is", sentence)
  sentence = re.sub(r"\'d", " would", sentence)
  sentence = re.sub(r"\'ll", " will", sentence)
  sentence = re.sub(r"\'t", " not", sentence)
  sentence = re.sub(r"\'ve", " have", sentence)
  sentence = re.sub(r"\'m", " am", sentence)
  sentence = re.sub(r"wont", "will not", sentence)
  sentence = re.sub(r"wouldnt", "would not", sentence)
  sentence = re.sub(r"shouldnt", "should not", sentence)
  sentence = re.sub(r"couldnt", "could not", sentence)
  sentence = re.sub(r"cudnt", "could not", sentence)
  sentence = re.sub(r"cant", "can not", sentence)
  sentence = re.sub(r"dont", "do not", sentence)
  sentence = re.sub(r"doesnt", "does not", sentence)
  sentence = re.sub(r"didnt", "did not", sentence)
  sentence = re.sub(r"wasnt", "was not", sentence)
  sentence = re.sub(r"werent", "were not", sentence)
  sentence = re.sub(r"havent", "have not", sentence)
  sentence = re.sub(r"hadnt", "had not", sentence)
  sentence = re.sub(r"ain't", "am not",sentence)
  sentence = re.sub(r"aren't", "are not",sentence)
  sentence = re.sub(r"can't", "cannot",sentence)
  sentence = re.sub(r"can't've", "cannot have",sentence)
  sentence = re.sub(r"'cause", "because",sentence)
  sentence = re.sub(r"could've", "could have",sentence)
  sentence = re.sub(r"couldn't", "could not",sentence)
  sentence = re.sub(r"couldn't've", "could not have",sentence)
  sentence = re.sub(r"didn't", "did not",sentence)
  sentence = re.sub(r"doesn't", "does not",sentence)
  sentence = re.sub(r"don't", "do not",sentence)
  sentence = re.sub(r"hadn't", "had not",sentence)
  sentence = re.sub(r"hadn't've", "had not have",sentence)
  sentence = re.sub(r"hasn't", "has not",sentence)
  sentence = re.sub(r"haven't", "have not",sentence)
  sentence = re.sub(r"he'd", "he would",sentence)
  sentence = re.sub(r"he'd've", "he would have",sentence)
  sentence = re.sub(r"he'll", "he will",sentence)
  sentence = re.sub(r"he'll've", "he will have",sentence)
  sentence = re.sub(r"he's", "he is",sentence)
  sentence = re.sub(r"how'd", "how did",sentence)
  sentence = re.sub(r"how'd'y", "how do you",sentence)
  sentence = re.sub(r"how'll", "how will",sentence)
  sentence = re.sub(r"how's", "how is",sentence)
  sentence = re.sub(r"i'd", "I would",sentence)
  sentence = re.sub(r"i'd've", "I would have",sentence)
  sentence = re.sub(r"i'll", "I will",sentence)
  sentence = re.sub(r"i'll've", "I will have",sentence)
  sentence = re.sub(r"i'm", "I am",sentence)
  sentence = re.sub(r"i've", "I have",sentence)
  sentence = re.sub(r"isn't", "is not",sentence)
  sentence = re.sub(r"it'd", "it had",sentence)
  sentence = re.sub(r"it'd've", "it would have",sentence)
  sentence = re.sub(r"it'll", "it will",sentence)
  sentence = re.sub(r"it'll've", "it will have",sentence)
  sentence = re.sub(r"it's", "it is",sentence)
  sentence = re.sub(r"let's", "let us",sentence)
  sentence = re.sub(r"ma'am", "madam",sentence)
  sentence = re.sub(r"mayn't", "may not",sentence)
  sentence = re.sub(r"might've", "might have",sentence)
  sentence = re.sub(r"mightn't", "might not",sentence)
  sentence = re.sub(r"mightn't've", "might not have",sentence)
  sentence = re.sub(r"must've", "must have",sentence)
  sentence = re.sub(r"mustn't", "must not",sentence)
  sentence = re.sub(r"mustn't've", "must not have",sentence)
  sentence = re.sub(r"needn't", "need not",sentence)
  sentence = re.sub(r"needn't've", "need not have",sentence)
  sentence = re.sub(r"o'clock", "of the clock",sentence)
  sentence = re.sub(r"oughtn't", "ought not",sentence)
  sentence = re.sub(r"oughtn't've", "ought not have",sentence)
  sentence = re.sub(r"shan't", "shall not",sentence)
  sentence = re.sub(r"sha'n't", "shall not",sentence)
  sentence = re.sub(r"shan't've", "shall not have",sentence)
  sentence = re.sub(r"she'd", "she would",sentence)
  sentence = re.sub(r"she'd've", "she would have",sentence)
  sentence = re.sub(r"she'll", "she will",sentence)
  sentence = re.sub(r"she'll've", "she will have",sentence)
  sentence = re.sub(r"she's", "she is",sentence)
  sentence = re.sub(r"should've", "should have",sentence)
  sentence = re.sub(r"shouldn't", "should not",sentence)
  sentence = re.sub(r"shouldn't've", "should not have",sentence)
  sentence = re.sub(r"so've", "so have",sentence)
  sentence = re.sub(r"so's", "so is",sentence)
  sentence = re.sub(r"that'd", "that would",sentence)
  sentence = re.sub(r"that'd've", "that would have",sentence)
  sentence = re.sub(r"that's", "that is",sentence)
  sentence = re.sub(r"there'd", "there had",sentence)
  sentence = re.sub(r"there'd've", "there would have",sentence)
  sentence = re.sub(r"there's", "there is",sentence)
  sentence = re.sub(r"they'd", "they would",sentence)
  sentence = re.sub(r"they'd've", "they would have",sentence)
  sentence = re.sub(r"they'll", "they will",sentence)
  sentence = re.sub(r"they'll've", "they will have",sentence)
  sentence = re.sub(r"they're", "they are",sentence)
  sentence = re.sub(r"they've", "they have",sentence)
  sentence = re.sub(r"to've", "to have",sentence)
  sentence = re.sub(r"wasn't", "was not",sentence)
  sentence = re.sub(r"we'd", "we had",sentence)
  sentence = re.sub(r"we'd've", "we would have",sentence)
  sentence = re.sub(r"we'll", "we will",sentence)
  sentence = re.sub(r"we'll've", "we will have",sentence)
  sentence = re.sub(r"we're", "we are",sentence)
  sentence = re.sub(r"we've", "we have",sentence)
  sentence = re.sub(r"weren't", "were not",sentence)
  sentence = re.sub(r"what'll", "what will",sentence)
  sentence = re.sub(r"what'll've", "what will have",sentence)
  sentence = re.sub(r"what're", "what are",sentence)
  sentence = re.sub(r"what's", "what is",sentence)
  sentence = re.sub(r"what've", "what have",sentence)
  sentence = re.sub(r"when's", "when is",sentence)
  sentence = re.sub(r"when've", "when have",sentence)
  sentence = re.sub(r"where'd", "where did",sentence)
  sentence = re.sub(r"where's", "where is",sentence)
  sentence = re.sub(r"where've", "where have",sentence)
  sentence = re.sub(r"who'll", "who will",sentence)
  sentence = re.sub(r"who'll've", "who will have",sentence)
  sentence = re.sub(r"who's", "who is",sentence)
  sentence = re.sub(r"who've", "who have",sentence)
  sentence = re.sub(r"why's", "why is",sentence)
  sentence = re.sub(r"why've", "why have",sentence)
  sentence = re.sub(r"will've", "will have",sentence)
  sentence = re.sub(r"won't", "will not",sentence)
  sentence = re.sub(r"won't've", "will not have",sentence)
  sentence = re.sub(r"would've", "would have",sentence)
  sentence = re.sub(r"wouldn't", "would not",sentence)
  sentence = re.sub(r"wouldn't've", "would not have",sentence)
  sentence = re.sub(r"y'all", "you all",sentence)
  sentence = re.sub(r"y'alls", "you alls",sentence)
  sentence = re.sub(r"y'all'd", "you all would",sentence)
  sentence = re.sub(r"y'all'd've", "you all would have",sentence)
  sentence = re.sub(r"y'all're", "you all are",sentence)
  sentence = re.sub(r"y'all've", "you all have",sentence)
  sentence = re.sub(r"you'd", "you had",sentence)
  sentence = re.sub(r"you'd've", "you would have",sentence)
  sentence = re.sub(r"you'll", "you you will",sentence)
  sentence = re.sub(r"you'll've", "you you will have",sentence)
  sentence = re.sub(r"you're", "you are",sentence)
  sentence = re.sub(r"you've", "you have",sentence)

  
  sentence = re.sub(r'[^\w\s]','',sentence) # Remove Punctutation
  
  sentence = sentence.lower() # Lower case
  
  CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  sentence = re.sub(CLEANR, '', sentence)# Remove HTML
  
  sentence = re.sub('[^A-Za-z]',' ',sentence)# Remove Digits
 
  sentence = re.sub(r'[\w._%+-]{1,20}@[\w.-]{1,20}.[A-Za-z]{2,3}','', sentence)# Remove Email
  
  sentence = re.sub(r'^http?s:\/\/.*[\r\n]*','', sentence)# Remove URLs
  
  sentence = re.sub(r'@[A-Za-z0-9]+','',sentence) #Remove Mentions  

  sentence = " ".join(item for item in sentence.split() if item not in stopwords.words('english') )
  
  sentence = ' '.join([PorterStemmer().stem(word)for word in sentence.split()])

  sentence = ' '.join([WordNetLemmatizer().lemmatize(word) for word in sentence.split()])
  return sentence

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

for text in df.articleBody:
    preprocessedText = preprocessing(text)
    sid = SentimentIntensityAnalyzer()
    polarity = sid.polarity_scores(str(preprocessedText))    
    tmpdic = {}    
    tmpdic.update(polarity)
    article_sentiments= pd.concat([article_sentiments,pd.DataFrame([tmpdic])], ignore_index=True)    
    
    


In [8]:
article_sentiments.to_csv("News_Data/article_sentiments_RI_2011-2023.csv", sep=',', encoding='utf-8', header=True)